In [1]:
import pandas as pd
pd.set_option("display.max_rows", 1000)

In [2]:
pop_df = pd.read_csv('./Resources/population_by_zip_2010.csv')

In [3]:
pop_df.shape

(1622831, 6)

In [4]:
pop_df['zipcode'].nunique()

33119

In [5]:
pop_df['gender'].iat[36]

nan

In [32]:
pop_df[(pop_df['minimum_age'].isnull())]

,population,minimum_age,maximum_age,gender,zipcode,geo_id


In [33]:
pop_df.dropna(subset=['minimum_age'],inplace=True)

In [34]:
pop_df[(pop_df['gender']=='NaN')]

,population,minimum_age,maximum_age,gender,zipcode,geo_id


In [35]:
pop_df.shape

(1523474, 6)

## Population Dataframe for upload to Postgres

In [36]:
population_df = pop_df[['population', 'minimum_age','maximum_age', 'gender', 'zipcode']].copy(deep=True)

In [37]:
population_df.head()

,population,minimum_age,maximum_age,gender,zipcode
0,50,30.0,34.0,female,61747
1,5,85.0,NaN,male,64120
2,1389,30.0,34.0,male,95117
3,231,60.0,61.0,female,74074
4,56,0.0,4.0,female,58042


In [43]:
from sqlalchemy import create_engine
conn = "postgres:Redcherry@localhost:5432/inpatient_charges_db"
engine = create_engine(f'postgresql://{conn}')


In [44]:
engine.table_names()

['population']

In [45]:
population_df.to_sql(name = 'population', con=engine, if_exists='append', index=False)

In [60]:
income_df = pd.read_csv('./Resources/kaggle_income.csv', encoding='latin-1')

In [61]:
income_df[income_df['Primary'] == 'place']['Zip_Code'].value_counts()

78584    10
78582     7
66749     4
95076     4
21502     4
         ..
60060     1
45723     1
53578     1
19096     1
77546     1
Name: Zip_Code, Length: 2863, dtype: int64

In [62]:
income_df = income_df[income_df['Primary'] == 'place']

## Income DataFrame for upload to Postgres

In [63]:
income_df = income_df[['State_Name','County','City','Place','Type','Zip_Code','Mean','Median','Stdev']]

In [64]:
income_df.head()

,State_Name,County,City,Place,Type,Zip_Code,Mean,Median,Stdev
0,Alabama,Mobile County,Chickasaw,Chickasaw city,City,36611,38773,30506,33101
1,Alabama,Barbour County,Louisville,Clio city,City,36048,37725,19528,43789
2,Alabama,Shelby County,Columbiana,Columbiana city,City,35051,54606,31930,57348
3,Alabama,Mobile County,Satsuma,Creola city,City,36572,63919,52814,47707
4,Alabama,Mobile County,Dauphin Island,Dauphin Island,Town,36528,77948,67225,54270


In [65]:
income_df[['Zip_Code', 'City']].sort_values('Zip_Code')
income_df.rename(columns={'Zip_Code':'zipcode'}, inplace=True)

In [66]:
income_df.head()

,State_Name,County,City,Place,Type,zipcode,Mean,Median,Stdev
0,Alabama,Mobile County,Chickasaw,Chickasaw city,City,36611,38773,30506,33101
1,Alabama,Barbour County,Louisville,Clio city,City,36048,37725,19528,43789
2,Alabama,Shelby County,Columbiana,Columbiana city,City,35051,54606,31930,57348
3,Alabama,Mobile County,Satsuma,Creola city,City,36572,63919,52814,47707
4,Alabama,Mobile County,Dauphin Island,Dauphin Island,Town,36528,77948,67225,54270


In [67]:
engine.table_names()

['income', 'population']

In [68]:
income_df.to_sql(name = 'income', con=engine, if_exists='append', index=False)

In [69]:
inpatient_df = pd.read_csv('./Resources/inpatientCharges.csv')

In [70]:
inpatient_df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


## Provider Dataframe

In [71]:
provider_df = inpatient_df[['Provider Id','Provider Name','Provider City','Provider State','Provider Zip Code']].copy(deep=True)
provider_df["Provider Id"].duplicated().sum()

159728

In [72]:
provider_df['Provider Id'].nunique()
provider_df.drop_duplicates(inplace=True)
# provider_df.duplicated()
provider_df.shape
provider_df.rename(columns={'Provider Zip Code':'zipcode'}, inplace=True)
provider_df.columns = provider_df.columns.str.replace(" ","")
provider_df.head()
# provider_df["Provider Id"].duplicated().sum()

,ProviderId,ProviderName,ProviderCity,ProviderState,zipcode
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301
1,10005,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631
3,10011,ST VINCENT'S EAST,BIRMINGHAM,AL,35235
4,10016,SHELBY BAPTIST MEDICAL CENTER,ALABASTER,AL,35007


In [73]:
engine.table_names()

['income', 'population', 'provider']

In [74]:
provider_df.to_sql(name = 'provider', con=engine, if_exists='append', index=False)

## InpatientCharges Dataframe

In [88]:
inpatientcharges_df = inpatient_df[['DRG Definition','Provider Id',' Total Discharges ', ' Average Covered Charges ',' Average Total Payments ', 'Average Medicare Payments']]
headers = [' Average Covered Charges ',' Average Total Payments ', 'Average Medicare Payments']
for header in headers:
    inpatientcharges_df.loc[:,header] = inpatientcharges_df.loc[:,header].str.replace("$","").astype(float)
# inpatientcharges_df.columns = inpatientcharges_df.columns.str.replace(" ","")
inpatientcharges_df.head()
# inpatientcharges_df[' Average Covered Charges ',' Average Total Payments ', 'Average Medicare Payments']

/Users/mohanpanga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,DRG Definition,Provider Id,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,91,32963.07,5777.24,4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,14,15131.85,5787.57,4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,24,37560.37,5434.95,4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,25,13998.28,5417.56,4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,18,31633.27,5658.33,4851.44


In [76]:
engine.table_names()

['income', 'population', 'provider', 'inpatient']

In [80]:
inpatientcharges_df.to_sql(name = 'inpatient', con=engine, if_exists='append', index=False)

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "$32963.07"
LINE 1: ...- EXTRACRANIAL PROCEDURES W/O CC/MCC', 10001, 91, '$32963.07...
                                                             ^

[SQL: INSERT INTO inpatient ("DRGDefinition", "ProviderId", "TotalDischarges", "AverageCoveredCharges", "AverageTotalPayments", "AverageMedicarePayments") VALUES (%(DRGDefinition)s, %(ProviderId)s, %(TotalDischarges)s, %(AverageCoveredCharges)s, %(AverageTotalPayments)s, %(AverageMedicarePayments)s)]
[parameters: ({'DRGDefinition': '039 - EXTRACRANIAL PROCEDURES W/O CC/MCC', 'ProviderId': 10001, 'TotalDischarges': 91, 'AverageCoveredCharges': '$32963.07', 'AverageTotalPayments': '$5777.24', 'AverageMedicarePayments': '$4763.73'}, {'DRGDefinition': '039 - EXTRACRANIAL PROCEDURES W/O CC/MCC', 'ProviderId': 10005, 'TotalDischarges': 14, 'AverageCoveredCharges': '$15131.85', 'AverageTotalPayments': '$5787.57', 'AverageMedicarePayments': '$4